In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from BVP_Analysis2 import run_on_video as rov
from BVP_Analysis2 import *
from tqdm import tqdm
folder_path = "Path"

# Get list of files in the folder
video_files = os.listdir(folder_path)

all_face_brightness = []
all_frame_brightness = []
all_average_psnr = []
all_mean_displacement = []
all_average_skin_tone = []
all_dist = []
freq_dif = []
all_name = []


def select_best_face(faces):
    if len(faces) == 0:
        return None
    
    # Select the largest face based on area
    best_face = max(faces, key=lambda x: x[2] * x[3])
    return best_face

def calculate_green_brightness(frame):
    # Extract the green channel
    green_channel = frame[:, :, 1]  # Green channel is at index 1 in OpenCV's BGR format

    # Calculate average brightness of the green channel
    brightness = int(green_channel.mean())
    return brightness

# Iterate through each file
for video_file in tqdm(video_files, desc="Processing", unit="file"):
    # Check if the file is a video file (assuming extension is ".mp4")
    if not video_file.endswith(".mp4"):
        # If it's not an MP4 file, skip to the next iteration
        continue
    
    # Construct paths for video and corresponding CSV file
    video_path = os.path.join(folder_path, video_file)
    csv_file = os.path.splitext(video_file)[0] + ".csv"
    csv_path = os.path.join(folder_path, csv_file)
    
    # Perform your analysis using video_path and csv_path
    # For example:
    #print("Processing:", video_path, csv_path)

    
    brightness_values = []
    brightness_values_all = []
    frames_read = 0

    # Initialize variables
    prev_frame = None
    psnr_values = []

    # Initialize feature detector
    feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)
    lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

    # Initialize variables
    prev_frame = None
    prev_pts = None
    mean_displacement = []

    skin_tones = []

    
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Couldn't open the video file.")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Detect faces in the frame
        faces = detect_faces(frame)

        # Select the best face
        best_face = select_best_face(faces)

        if best_face is not None:
            x, y, w, h = best_face
            face_roi = frame[y:y+h, x+int(w/6):x+int(w*(5/6))]
            brightness_frame = calculate_green_brightness(face_roi)
            brightness_frame_all = calculate_brightness(frame)
            brightness_values.append(brightness_frame)
            brightness_values_all.append(brightness_frame_all)

            hsv_face = cv2.cvtColor(face_roi, cv2.COLOR_BGR2HSV)

            # Define lower and upper bounds for skin color in HSV
            lower_skin = np.array([0, 48, 80], dtype=np.uint8)
            upper_skin = np.array([20, 255, 255], dtype=np.uint8)

            # Mask skin pixels
            skin_mask = cv2.inRange(hsv_face, lower_skin, upper_skin)

            # Calculate skin tone metric (e.g., mean of hue channel)
            if np.sum(skin_mask) > 0:
                skin_tone = np.mean(hsv_face[..., 0][skin_mask > 0])

                # Store skin tone metric
                skin_tones.append(skin_tone)


        
        # Convert frame to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        if prev_frame is not None:
            # Calculate PSNR between consecutive frames
            psnr = cv2.PSNR(prev_frame, gray_frame)
            psnr_values.append(psnr)


        if prev_frame is not None and prev_pts is not None:
            # Calculate optical flow
            next_pts, status, err = cv2.calcOpticalFlowPyrLK(prev_frame, gray_frame, prev_pts, None, **lk_params)

            # Filter out points with low status
            good_new = next_pts[status == 1]
            good_prev = prev_pts[status == 1]

            # Calculate displacement
            displacement = cv2.norm(good_new - good_prev)

            mean_displacement.append(displacement)

        # Update previous frame and points
        prev_frame = gray_frame
        prev_pts = cv2.goodFeaturesToTrack(prev_frame, mask=None, **feature_params)


        frames_read += 1

    cap.release()


    #bright_face = np.mean(brightness_values)
    bright_face = brightness_values[15]
    #print("Face Brightness:", bright_face)

    #bright_frame = np.mean(brightness_values_all)
    bright_frame = brightness_values_all[15]
    #print("frame Brightness:", bright_frame)

    # Calculate average PSNR value
    average_psnr = sum(psnr_values) / len(psnr_values)
    #print("Average PSNR:", average_psnr)
            

    mean_displacement = sum(mean_displacement) / len(mean_displacement)
    #print("Mean Displacement:", mean_displacement)


    average_skin_tone = np.mean(skin_tones)
    #print("Average Skin Tone:", average_skin_tone)

    # Store the features for this video
    all_face_brightness.append(np.mean(brightness_values))
    all_frame_brightness.append(np.mean(brightness_values_all))
    all_average_psnr.append(np.mean(psnr_values))
    all_mean_displacement.append(np.mean(mean_displacement))
    all_average_skin_tone.append(np.mean(skin_tones))
    all_name.append(video_file)

   
   
     


    new_label = video_path
    recordingtime = 35
    method = "cpu_GREEN"
    roi_approach = 'hol'
    roi_method = "convexhull"
    pre_filt = False
    post_filt = False

    times, BPM, uncertainty, bvps = rov(video_path,cuda=False, roi_approach=roi_approach, roi_method=roi_method,pre_filt=pre_filt, post_filt=post_filt, method = method)
    peaks1, peaks2, peaks_rppg, y1, y2, x1, x2, path_norm, dist = analysis(video_path, csv_path, bvps)

    all_dist.append(dist)

    y1 = y1 - np.mean(y1)
    fft_y1 = np.fft.fft(y1)
    frequencies_y1 = np.fft.fftfreq(len(fft_y1), d=(1/25 ))  # Frequency values


    positive_freq_mask = frequencies_y1 >= 0
    frequencies_y1 = frequencies_y1[positive_freq_mask]
    fft_y1 = fft_y1[positive_freq_mask]

    y2 = y2 - np.mean(y2)
    fft_y2 = np.fft.fft(y2)
    frequencies_y2 = np.fft.fftfreq(len(fft_y2), d=(1/30 ))  # Frequency values


    positive_freq_mask = frequencies_y2 >= 0
    frequencies_y2 = frequencies_y2[positive_freq_mask]
    fft_y2 = fft_y2[positive_freq_mask]

    max_index_y1 = np.argmax(np.abs(fft_y1))
    max_index_y2 = np.argmax(np.abs(fft_y2))
    max_freq_y1 = frequencies_y1[max_index_y1]
    max_freq_y2 = frequencies_y2[max_index_y2]

    diffreq = np.absolute(max_freq_y2-max_freq_y1)

    freq_dif.append(diffreq)

    #print("Distance:", dist)
    #print("freq Diff:", diffreq)



    
# Convert lists to numpy arrays
all_face_brightness = np.array(all_face_brightness)
all_frame_brightness = np.array(all_frame_brightness)
all_average_psnr = np.array(all_average_psnr)
all_mean_displacement = np.array(all_mean_displacement)
all_average_skin_tone = np.array(all_average_skin_tone)
all_dist = np.array(all_dist)
freq_dif = np.asarray(freq_dif)
all_name = np.asarray(all_name)

print("Processed all segment")

Processing: 100%|██████████| 40/40 [05:48<00:00,  8.71s/file]

Processed all segment


In [2]:
np.save('all_face_brightness.npy', all_face_brightness)
np.save('all_frame_brightness.npy', all_frame_brightness)
np.save('all_average_psnr.npy', all_average_psnr)
np.save('all_mean_displacement.npy', all_mean_displacement)
np.save('all_average_skin_tone.npy', all_average_skin_tone)
np.save('all_dist.npy', all_dist)
np.save('freq_dif.npy', freq_dif)
np.save('all_name.npy', all_name)